# 머신러닝 기반 분류기
머신러닝으로 이를 학습하고 예측하는 모델이다.
tfidf - SVC 와 CountVec - MultinomialNB 두 조합이 사용되었다.

### 데이터 불러오고 전처리
whole = 전체 데이터
df = test용 데이터

In [1]:
import pandas as pd
whole = pd.read_csv("data.csv")
df = pd.read_csv("tmp.csv")

In [2]:
whole.groupby(whole['별점']).size()

별점
1     734
2     503
3    1134
4    1346
5    1784
dtype: int64

In [3]:
len(whole)

5501

In [4]:
from konlpy.tag import Okt
okt = Okt()

In [5]:
def clean_text(text):
    text = okt.pos(text, norm = True, stem = True)
    tmp = []
    conti = False
    for i in text:           
        if  i[1] == "Adjective" or i[1] == "Noun" or i[1] == "Verb" or i[1] == "Adjective" or i[1] == "Suffix" or i[1] =="KoreanParticle":
            tmp.append(i[0])
            prev = (i[0], i[1])
        
    
    return " ".join(tmp)

In [6]:
whole["전처리"] = whole["강의평"].map(clean_text)
df["전처리"] = df["강의평"].map(clean_text)

In [8]:
# whole_pn = whole[whole["긍정부정"] =="neg"].append(whole[whole["긍정부정"] =="pos"])
# df_pn = df[df["긍정부정"] =="neg"].append(df[df["긍정부정"] =="pos"])

In [7]:
train_text = list(whole["전처리"])
train_sent = list(whole["별점"])
test_text = list(df["전처리"])
test_sent = list(df["별점"])

### tfidf와 SVC를 사용한 모델

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
term_docs_train = tfidf_vectorizer.fit_transform(train_text)
term_docs_test = tfidf_vectorizer.transform(test_text)

In [10]:
from sklearn.svm import LinearSVC
svm = LinearSVC(C= 1.0, penalty = "l2", random_state= 42)
svm.fit(term_docs_train, train_sent)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [11]:
from sklearn.metrics import classification_report
def evaluate(test_x, test_y, model):
    predictions = model.predict(test_x)
    print(classification_report(test_y, predictions))
#     print("Macro F1 score - {0:0.5f}".format(classification_report(test_y, predictions, output_dict=True)["macro avg"]["f1-score"]))
evaluate(term_docs_test, test_sent, svm)
accuracy = svm.score(term_docs_test, test_sent)

print("accuracy: %0.3f"%(accuracy*100),"%")

              precision    recall  f1-score   support

           1       0.53      0.34      0.42        29
           2       0.13      0.14      0.14        14
           3       0.48      0.26      0.34        42
           4       0.36      0.41      0.38        32
           5       0.26      0.61      0.37        18

    accuracy                           0.35       135
   macro avg       0.35      0.35      0.33       135
weighted avg       0.40      0.35      0.35       135

accuracy: 34.815 %


### CountVectorizer와 MultinomialNB를 사용한 모델

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(lowercase=True, max_df=1.0)
term_docs_train_s = CV.fit_transform(train_text)
term_docs_test_s = CV.transform(test_text)

In [36]:
#최적의 결과를 바탕으로 설정해줌
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha = 0.25, fit_prior=False)
nb.fit(term_docs_train_s, train_sent)

MultinomialNB(alpha=0.25, class_prior=None, fit_prior=False)

In [37]:
from sklearn.metrics import classification_report
def evaluate(test_x, test_y, model):
    predictions = model.predict(test_x)
    print(classification_report(test_y, predictions))
#     print("Macro F1 score - {0:0.5f}".format(classification_report(test_y, predictions, output_dict=True)["macro avg"]["f1-score"]))
evaluate(term_docs_test_s, test_sent, nb)
accuracy = nb.score(term_docs_test_s, test_sent)

print("accuracy: %0.3f"%(accuracy*100),"%")

              precision    recall  f1-score   support

           1       0.71      0.52      0.60        29
           2       0.29      0.29      0.29        14
           3       0.36      0.21      0.27        42
           4       0.34      0.53      0.41        32
           5       0.36      0.50      0.42        18

    accuracy                           0.40       135
   macro avg       0.41      0.41      0.40       135
weighted avg       0.42      0.40      0.40       135

accuracy: 40.000 %


In [8]:
from xgboost import plot_importance
from xgboost import XGBClassifier

In [14]:
xgb = XGBClassifier(n_estimators= 500, learning_rate= 0.1, max_depth = 4)
xgb.fit(term_docs_train, train_sent)
xgb_pred = xgb.predict(term_docs_test)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(test_sent, xgb_pred))
accuracy = xgb.score(term_docs_test, test_sent)
print("accuracy: %0.3f"%(accuracy*100),"%")

              precision    recall  f1-score   support

           1       0.71      0.41      0.52        29
           2       0.14      0.07      0.10        14
           3       0.39      0.31      0.35        42
           4       0.30      0.34      0.32        32
           5       0.22      0.50      0.31        18

    accuracy                           0.34       135
   macro avg       0.35      0.33      0.32       135
weighted avg       0.39      0.34      0.35       135

accuracy: 34.074 %


### 위에 report는 완벽하게 이 모델의 성능을 증명하지 못한다.

왜냐하면 5점이여야 하는 것을 4점으로 평가하든 1점으로 평가하든 모두 틀렸다고만 평가할 수 있기 때문이다.

그러므로 제대로된 평가를 하려면 실제 라벨링된 것과 예측한 라벨링의 오차(cost)가 중요하다. 그러므로 오차값의 평균으로 오차를 평가한다.

cost = avg(abs(real - estimated))

In [38]:
mean = 0
for i in range(len(test_sent)):
    mean += abs(test_sent[i]- nb.predict(term_docs_test_s))

(mean/len(test_sent)).sum()/len(test_sent)

1.4996982167352535

In [32]:
mean = 0
for i in range(len(test_sent)):
    mean += abs(test_sent[i]- svm.predict(term_docs_test))

(mean/len(test_sent)).sum()/len(test_sent)

1.5866666666666667

In [17]:
mean = 0
for i in range(len(test_sent)):
    mean += abs(test_sent[i]- xgb.predict(term_docs_test))

(mean/len(test_sent)).sum()/len(test_sent)

1.5458984910836762

이를 통해 일반적으로 데이터를 넣었을 때, 좋은 성능에서 1.5점 정도의 오차값을 보인다고 할 수 있다. 즉 실제로 4점인 데이터를 넣었을 때, 2.5 ~ 5점 으로 예측한다는 것이다. 더 많은 데이터를 학습시키면 그 격차는 줄 것이다. 딥러닝을 사용한 모델은 훨씬 더 정확하게 예측하였다.

### 실제 데이터를 넣어서 예측

In [40]:
a = ["아 개같은 수업 못들어 먹겠다."]
a_test = tfidf_vectorizer.transform(a)
nb.predict(a_test)

array([1])

In [41]:
a = ["정말 최고의 수업입니다."]
a_test = tfidf_vectorizer.transform(a)
nb.predict(a_test)

array([5])